In [163]:
from collections import defaultdict, deque
import random
import math
import time
import random
import numpy as np

class City:
    id = 0
    def __init__(self):
        self.X = random.randint(-100, 100)
        self.Y = random.randint(-100, 100)
        self.Z = random.randint(0, 50)
        self.coordinates = np.array((self.X, self.Y, self.Z))
        self.id = City.id
        City.id += 1
    
class Graph:
    
    def __init__(self, cities):
        self.all_paths = []
        self.distance_list = []
        self.adjList = defaultdict(list)
        self.sym_matrix = self.generate_sym_matrix(cities)
        self.matrix_80 = self.generate_80_matrix(cities)
        self.asym100_matrix = self.generate_asym_matrix(cities)
        self.asym80_matrix = self.generate_80asym_matrix(cities)

    #Stworzenie symetrycznej macierzy 100%

    def generate_sym_matrix(self, cities):
        sym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            sym_matrix.append(row)
        return np.array(sym_matrix)
    

    #Stworzenie symetrycznej macierzy 80%

    def generate_80_matrix(self, cities):
        matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            matrix.append(row)
        matrix = np.array(matrix)
        total_elements = matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(matrix.shape)
    

    #Stworzenie asymetrycznej macierzy 100%

    def generate_asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        return np.array(asym_matrix)
    

    #Stworzenie asymetrycznej macierzy 80%

    def generate_80asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        asym_matrix = np.array(asym_matrix)
        total_elements = asym_matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = asym_matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1 or matrix_flattened[index] == 2:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(asym_matrix.shape)


    #Metoda obliczajaca odleglosc bez uwzglednienia asymetrycznosci
    def calculate_distance(self, city1, city2):
        return np.linalg.norm(city1.coordinates - city2.coordinates)
    


    #Metoda obliczajaca odleglosc z uwzglednieniem asymetrycznosci
    def calculate_distance_asym(self, city1, city2):
        if city1.Z < city2.Z:
            #print("Going up")
            return 1.1 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        elif city1.Z > city2.Z:
            #print("Going down")
            return 0.9 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        else:
            #print("Same height")
            return np.linalg.norm(city1.coordinates - city2.coordinates)


    #Dodawanie krawedzi do grafu (do zmiennej adjList)
    def addEdge(self, u, v):
            self.adjList[u].append(v)

    #Definiowanie krawedzi i przekazanie do metody addEdge
    def createEdge(self, matrix):
        for i in range(0, len(matrix)):
            for j in range(0, len(matrix)):
                if matrix[i][j] == 1 or matrix[i][j] == 2:
                    self.addEdge(i, j)
                    #print(f"Created edge between {i} and {j}")

                    
    #Metoda obliczajaca wszystkie mozliwe sciezki metoda BFS
    def bfs(self, startNode):
        queue = deque([(startNode, [startNode])])
        paths = []

        while queue:
            currentNode, path = queue.popleft()
            #print(f"BFS: {path}")
            
            if len(path) == len(self.adjList) + 1:
                paths.append(path)
                # print(path)
                continue

            for neighbour in self.adjList[currentNode]:
                if neighbour not in path:
                    queue.append((neighbour, path + [neighbour]))
                elif len(path) == len(self.adjList) and neighbour == 0:
                    queue.append((neighbour, path + [neighbour]))
        return paths
    
    #Metoda obliczajaca wszystkie mozliwe sciezki metoda DFS
    def dfs(self, startNode):
        queue = deque([(startNode, [startNode])])
        paths = []

        while queue:
            currentNode, path = queue.pop()
            #print(f"DFS: {path}")
            
            if len(path) == len(self.adjList) + 1:
                paths.append(path)
                # print(path)
                continue

            for neighbour in self.adjList[currentNode]:
                if neighbour not in path:
                    queue.append((neighbour, path + [neighbour]))
                elif len(path) == len(self.adjList) and neighbour == 0:
                    # print(f'{len(path)} == {len(self.adjList)}')
                    # print(f'AdjList: {self.adjList}')
                    queue.append((neighbour, path + [neighbour]))
        return paths
    
    #Wersja dla 1 NN
    def NN(self, startNode, sym, cities):
        path = [startNode]
        visited = set()
        visited.add(startNode)
        distance = 0
        currentNode = startNode

        while len(path) < len(self.adjList) + 1:
            closest_distance = float('inf')
            closest_neighbor = None
            
            for neighbour in self.adjList[currentNode]:
                if neighbour not in visited:
                    if sym == 'sym':
                        dist = self.calculate_distance(cities[currentNode], cities[neighbour])
                    elif sym == 'asym':
                        dist = self.calculate_distance_asym(cities[currentNode], cities[neighbour])

                    if dist < closest_distance:
                        closest_distance = dist
                        closest_neighbor = neighbour


                elif len(path) == len(self.adjList) and neighbour == 0:
                    path.append(neighbour)
                    if sym == 'sym':
                        dist = self.calculate_distance(cities[currentNode], cities[neighbour])
                    elif sym == 'asym':
                        dist = self.calculate_distance_asym(cities[currentNode], cities[neighbour])
                    distance += dist
                    continue
            
            if closest_neighbor is not None:
                path.append(closest_neighbor)
                distance += closest_distance
                visited.add(closest_neighbor)
                currentNode = closest_neighbor
            else:
                # print(f'{currentNode} has no neighoburs')
                break
        return path, distance

    def NN2(self, startNode, sym, cities):
        path = [startNode]
        visited = set()
        dist = []
        visited.add(startNode)
        distance = 0
        currentNode = []
        neighbours = []
        currentNode.append(startNode)

        while len(path) < len(self.adjList) + 1:
            #print(f'Path len:{len(path)}, {len(self.adjList)}')
            closest_distance = float('inf')
            closest_neighbor = None
            
            for i in range(0, len(currentNode)):
                # print(f'\033[91mCurrent node: {currentNode}, {i}, Len: {len(currentNode)}\033[0m')
                for neighbour in self.adjList[currentNode[i]]:
                    if neighbour not in visited:
                        if sym == 'sym':
                            dist.append([self.calculate_distance(cities[currentNode[i]], cities[neighbour]), neighbour])
                            neighbours.append(neighbour)
                        elif sym == 'asym':
                            dist.append([self.calculate_distance_asym(cities[currentNode[i]], cities[neighbour]), neighbour])
                            neighbours.append(neighbour)
                        # print(f'added: {neighbour}')


                    elif len(path) == len(self.adjList) and neighbour == 0:
                        # print(f'Dodatkowe, {path}')
                        path.append(neighbour)
                        if sym == 'sym':
                            dist.append(self.calculate_distance(cities[currentNode[i]], cities[neighbour]))
                        elif sym == 'asym':
                            dist.append(self.calculate_distance_asym(cities[currentNode[i]], cities[neighbour]))
                        distance = np.float64(distance)
                        distance += dist
                        break
                    # print(f'\033[92mNieghbours: {neighbours}\033[0m')
                    



                if len(neighbours) == 0 and len(path) < len(self.adjList):
                    print(f'Bad path: {path}')
                    print(len(path), len(self.adjList), len(path) != len(self.adjList))
                    print(f'Error, {self.adjList[currentNode[i]]}')
                    path.remove(currentNode[0])
                    visited.remove(currentNode[0])
                    path.append(currentNode[1])
                    visited.add(currentNode[1])
                    # print(f'Path: {path}, visited: {visited}')
                neighbours.clear()

                if len(dist) != 0:
                    # print(f'is not None')
                    break

            if len(path) < len(self.adjList):
                # print(dist, type(dist))
                dist = sorted(dist, key=lambda row: row[0])
                # print(f'Distance: {dist}')
                closest_distance = dist[0][0]
                # if len(path) < len(self.adjList) - 1:
                #     closest_neighbor = [dist[0][1], dist[1][1]]
                if len(dist) >= 2:
                    closest_neighbor = [dist[0][1], dist[1][1]]
                else:
                    closest_neighbor = [dist[0][1]]
                dist.clear()
            
            if closest_neighbor is not None:
                path.append(closest_neighbor[0])
                # print(f'Path: {path}')
                distance += closest_distance
                visited.add(closest_neighbor[0])
                currentNode = closest_neighbor
            else:
                # print(f'{currentNode} has no neighoburs')
                break
        return path, distance



    
    #Metoda obliczajaca najkrotsza droge uwzgledniajac jedynie symetryczne grafy oraz metode wyszukiwania
    def find_shortest_sym_path(self, cities, method):
        min_distance = float('inf')
        path = float('inf')
        distance = 0
        return_path = []
        paths = []
        if method == 'bfs':
            print('\nBFS')
            paths = self.bfs(0)
        elif method == 'dfs':
            print('\nDFS')
            paths = self.dfs(0)
        #print(f"Paths: {paths}\n\n")
        for i in range(0, len(paths)):
            #print(f"{i}: {paths[i]}")
            for j in range(0, len(paths[i]) - 1):
                #print(f"({paths[i][j]}{paths[i][j + 1]})")
                distance += self.calculate_distance(cities[paths[i][j]], cities[paths[i][j+1]])
            #print(f"Distance in path {paths[i]}: {distance}")
            if distance < min_distance:
                return_path = paths[i]
                min_distance = distance
                path = i + 1
            distance = 0
        return f'Min distance is: {min_distance} for path {return_path}'
    

    #Metoda obliczajaca najkrotsza droge uwzgledniajac jedynie asymetryczne grafy oraz metode wyszukiwania
    def find_shortest_asym_path(self, cities, method):
        min_distance = float('inf')
        path = float('inf')
        distance = 0
        return_path = []
        paths = []
        if method == 'bfs':
            print('\nBFS')
            paths = self.bfs(0)
        elif method == 'dfs':
            print('\nDFS')
            paths = self.dfs(0)
        #print(f"Paths: {paths}\n\n")
        for i in range(0, len(paths)):
            #print(f"{i}: {paths[i]}")
            for j in range(0, len(paths[i]) - 1):
                #print(f"({paths[i][j]}{paths[i][j + 1]})")
                distance += self.calculate_distance_asym(cities[paths[i][j]], cities[paths[i][j+1]])
            #print(f"Distance in path {paths[i]}: {distance}")
            if distance < min_distance:
                return_path = paths[i]
                min_distance = distance
                path = i + 1
            distance = 0
        return f'Min distance is: {min_distance} for path {return_path}'
    
    #Metoda obliczajaca heurystyki
    def heuristic(self, cities, sym):
        h_dop = float('inf') #Min odlegosc
        h_niedop = 0 #Srednia odleglosc
    
        current_dist = 0
        divide = 0
        for i in range(len(cities)):
            for j in self.adjList[i]:
                if sym == 'sym':
                    current_dist = self.calculate_distance(cities[i], cities[j])
                elif sym == 'asym':
                    current_dist = self.calculate_distance_asym(cities[i], cities[j])
                h_niedop += current_dist
                # print(f'Current distance between {i} and {j}: {current_dist}')

                divide += 1

                if current_dist < h_dop:
                    # print(f'{current_dist} is lower than {h_dop}')
                    h_dop = current_dist
        
        h_niedop = h_niedop / divide
        # print(f'h_niedop = {h_niedop}, h_dop = {h_dop}')
        return h_dop, h_niedop
                
    #Metoda obliczajaca najkrotsza trase metoda A*
    def astar(self, startNode, cities, sym):
        h_dop, h_niedop = self.heuristic(cities, sym)
        queue = deque([(0, startNode, [startNode], 0)])
        paths = []
        h_chosen = h_dop
        total_distance = 0
        print(f'\nA*\nHeuristic: {h_chosen}')

        while queue:
            # print(f'\nBefore: {queue}')
            queue = deque(sorted(queue, key=lambda row: row[0]))
            # print(f'After: {queue}')
            total_cost, currentNode, path, distance_passed = queue.popleft()
            # print(f'Popped cost: {total_cost}, path: {path}\n')

            # print(path)
            
            if len(path) == len(self.adjList):
                paths.append(path)
                # print(path)
                continue

            for neighbour in self.adjList[currentNode]:
                if neighbour not in path:
                    if sym == 'sym':
                        actual_cost = self.calculate_distance(cities[currentNode], cities[neighbour])
                    elif sym == 'asym':
                        actual_cost = self.calculate_distance_asym(cities[currentNode], cities[neighbour])
                    g_n = distance_passed + actual_cost
                    h_n = (6-len(path)) * h_chosen
                    # print(f'Heuristic from {neighbour}: {h_n}, G_n cost: {g_n}, toal cost: {total_cost}, actual cost: {actual_cost}')
                    f_cost = g_n + h_n
                    queue.append((f_cost, neighbour, path + [neighbour], g_n))
                elif len(path) == len(self.adjList) - 1 and neighbour == 0:
                    path.append(0)
                    paths.append(path)
                    print(path)
                    queue.clear()
        paths = paths[0]
        for i in range(0, len(paths) - 1):
            if sym == 'sym':
                total_distance += self.calculate_distance(cities[paths[i]], cities[paths[i + 1]])
            elif sym == 'asym':
                total_distance += self.calculate_distance_asym(cities[paths[i]], cities[paths[i + 1]])
        return paths, total_distance


    def ACO(self, cities):
        ants = 50
        epochs = 100
        fx = len(cities)
        f = [0] * ants
        ants_path = []
        ants_path = self.initial_scan(ants, ants_path, fx, f)

        return ants_path
    
    def initial_scan(self, ants, ants_path, fx, f):
        currentNode = 0
        for ant in range(ants):
            print(f'Ant {ant}')
            i = 0
            currentNode = 0
            path = [[0]] * ants
            print(ants_path)
            print(path[i])

            while len(path[i]) < fx:
                print(f'Ants path: {len(path[i])}, fx: {fx}')
                if len(self.adjList[currentNode]) != 0:
                    # neighbour = np.random.choice(self.adjList[currentNode], p=[0.1, 0.1, 0.1, 0.7])
                    neighbour = np.random.choice(self.adjList[currentNode])
                    print(f'Current node: {currentNode}, current neighoburs {self.adjList[currentNode]}, choice: {neighbour}')
                    # neighbour = self.adjList[currentNode][random_choice]
                    # print(f'Neighbour: {neighbour}')
                else: continue
                
                if len(path[i]) == (fx - 1) and neighbour == 0:
                    path[i].append(neighbour)
                    break
            

                if neighbour not in path[i]:
                    path[i].append(neighbour)
                    print(f'Neighbour node before: {neighbour})')
                    currentNode = neighbour
                    print(f'Current node after: {currentNode})')
                    print(path[i])
            i += 1
            print(f'Path: {path}')
            ants_path.append(path[0])
                

        return ants_path


In [166]:
cities = []
for i in range(0, 11):
    cities.append(City())
    # print(f'City {i}: X: {cities[i].X}, Y: {cities[i].Y}, Z: {cities[i].Z}')
graph = Graph(cities)

#print(graph.sym_matrix)
#print(graph.matrix_80)
# print(graph.asym100_matrix)
#print(graph.asym80_matrix)

graph.createEdge(graph.sym_matrix)
# graph.createEdge(graph.matrix_80)
# graph.createEdge(graph.asym100_matrix)
# graph.createEdge(graph.asym80_matrix)

start_time = time.perf_counter()
min_distance_bfs = graph.find_shortest_sym_path(cities, 'dfs')
print(min_distance_bfs)
end_time = time.perf_counter()
print(f'DFS time: {(end_time - start_time)*100}')


# start_time = time.perf_counter()
# min_distance_bfs = graph.find_shortest_sym_path(cities, 'bfs') 
# print(min_distance_bfs)
# end_time = time.perf_counter()
# print(f'BFS time: {(end_time - start_time)*100}')

# print('\nNN')
# start_time = time.perf_counter()
# NN_path, NN_distance = graph.NN(0, 'sym', cities)
# print(f'Path: {NN_path}, distance: {NN_distance}')
# end_time = time.perf_counter()
# print(f'NN time: {(end_time - start_time)*100}')

# print('\nkNN')
# start_time = time.perf_counter()
# NN_path, NN_distance = graph.NN2(0, 'sym', cities)
# print(f'Path: {NN_path}, distance: {NN_distance}')
# end_time = time.perf_counter()
# print(f'kNN, k = 2 time: {(end_time - start_time)*100}')


# start_time = time.perf_counter()
# best_path, best_distance = graph.astar(0, cities, 'sym')
# print(f'Best path: {best_path}, best dist: {best_distance}')
# end_time = time.perf_counter()
# print(f'A* time: {(end_time - start_time)*100}')


start_time = time.perf_counter()
print(graph.ACO(cities))
end_time = time.perf_counter()
print(f'ACO time: {(end_time - start_time)*100}')


DFS
Min distance is: 555.6055398336133 for path [0, 7, 5, 1, 9, 6, 2, 4, 3, 8, 0]
DFS time: 786.954440001864
Ant 0
[]
[0]
Ants path: 1, fx: 11
Current node: 0, current neighoburs [1, 2, 3, 4, 5, 6, 7, 8, 9], choice: 2
Neighbour node before: 2)
Current node after: 2)
[0, 2]
Ants path: 2, fx: 11
Current node: 2, current neighoburs [0, 1, 3, 4, 5, 6, 7, 8, 9], choice: 0
Ants path: 2, fx: 11
Current node: 2, current neighoburs [0, 1, 3, 4, 5, 6, 7, 8, 9], choice: 5
Neighbour node before: 5)
Current node after: 5)
[0, 2, 5]
Ants path: 3, fx: 11
Current node: 5, current neighoburs [0, 1, 2, 3, 4, 6, 7, 8, 9], choice: 4
Neighbour node before: 4)
Current node after: 4)
[0, 2, 5, 4]
Ants path: 4, fx: 11
Current node: 4, current neighoburs [0, 1, 2, 3, 5, 6, 7, 8, 9], choice: 9
Neighbour node before: 9)
Current node after: 9)
[0, 2, 5, 4, 9]
Ants path: 5, fx: 11
Current node: 9, current neighoburs [0, 1, 2, 3, 4, 5, 6, 7, 8], choice: 3
Neighbour node before: 3)
Current node after: 3)
[0, 2, 5, 